In [1]:
from scipy.spatial.distance import pdist, cdist, squareform
import math
import pandas as pd
import numpy as np
from numba import cuda
from numba import jit
from sklearn import datasets
import cupy as cp

In [2]:
a = np.array([[1, 1], [2, 2], [3, 3]])
b = np.array([[5,5,5]]).reshape(-1,1)

a = cp.asarray(a)
b = cp.asarray(b)
c = cp.concatenate((b,a), axis = 1)
c = cp.asnumpy(c)

print(c)
print(c.shape)

[[5 1 1]
 [5 2 2]
 [5 3 3]]
(3, 3)


In [3]:
d = np.concatenate((b,a), axis = 1)
print(d)

[[5 1 1]
 [5 2 2]
 [5 3 3]]


In [4]:
def argwhere(data):
    aux1 = cp.nonzero(data)
    aux2 = aux1[0].reshape(-1,1)
    
    for i in range (1,len(aux1)):
        aux2 = cp.concatenate((aux1[i].reshape(-1,1),aux2), axis = 1)
    
    return aux2

In [5]:
x = np.array([[1,2,3],[4,5,6],[4,5,6],[4,5,6]])
print(x)

z = cp.asarray(x)
print(z)

[[1 2 3]
 [4 5 6]
 [4 5 6]
 [4 5 6]]
[[1 2 3]
 [4 5 6]
 [4 5 6]
 [4 5 6]]


In [6]:
a = np.argwhere(x)
print(a)

a = argwhere(z)
print(a)

[[0 0]
 [0 1]
 [0 2]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]
 [2 2]
 [3 0]
 [3 1]
 [3 2]]
[[0 0]
 [1 0]
 [2 0]
 [0 1]
 [1 1]
 [2 1]
 [0 2]
 [1 2]
 [2 2]
 [0 3]
 [1 3]
 [2 3]]


In [7]:
b = np.argwhere(np.isnan(x))
print(b)

b = argwhere(cp.isnan(z))
print(b)

[]
[]


In [99]:
def cp_pi_calculator(Uniquesample):
    UN, W = Uniquesample.shape
    #Xnorm = cp.matrix(cp.sqrt(cp.sum(cp.power(Uniquesample,2),axis=1))).T
    Xnorm = cp.sqrt(cp.sum(cp.power(Uniquesample,2),axis=1)).reshape(-1,1).T
    aux2 = Xnorm.T
    for i in range(W-1):
        aux2 = cp.concatenate((Xnorm.T,aux2),axis=1)
        
    #aux2 = cp.insert(aux2,0,Xnorm.T,axis=1)
    Uniquesample1 = Uniquesample / aux2
    AA2 = cp.mean(Uniquesample1,0).reshape(1,-1)
    X2 = 1
    DT2 = X2 - cp.sum(cp.power(AA2,2))
    aux = []
    aux2 = cp.empty((UN,1,W))
    for i in range(UN): aux.append(AA2)
    
    for i in range(UN): aux2 [i] = Uniquesample1[i]-aux[i]
    
    a = cp.power(aux2,2)
    b = cp.sum(a,axis=1)
    c = cp.sum(b,axis=1)
    uspi = c+DT2
        
    return uspi

In [100]:
c = cp_pi_calculator(z)

print(c)

[0.03805223 0.01268408 0.01268408 0.01268408]


In [101]:
def my_cdist_func(A, B):
    LA, WA = A.shape
    LB, WB = B.shape
    
    ret_array = cp.zeros((LA,LB))
    for i in range(LA):
        aux = cp.asarray([A[i]-B[j] for j in range(LB)])
        ret_array[i] = cp.sqrt(cp.sum(cp.power(aux,2),axis=1))
    
    return ret_array

In [2]:
import numpy as np

In [2]:
a = cp.asarray([1,2,3,4,5])

a

array([1, 2, 3, 4, 5])

In [3]:
b = cp.asarray([[1,2,3,4,5],
              [6,7,8,9,0]])

b

array([[1, 2, 3, 4, 5],
       [6, 7, 8, 9, 0]])

In [4]:
c = cp.vstack((a,b))

c

array([[1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [6, 7, 8, 9, 0]])

In [7]:
cp.amax(a) + 1

6